In [1]:
from main import CollocateTokenSelector, BERTReplacer, BERTTokenSelector
from main import Humourizer

import pandas as pd

import numpy as np

In [3]:
token_selector = CollocateTokenSelector('CM_SpaCy_truecased')
token_replacer = BERTReplacer(bert_model_path='bert_masked_lm_full_model',
                             bert_tokenizer="bert-large-uncased-whole-word-masking",
                             verbose=True, k=3)
humourizer = Humourizer(token_selector, token_replacer, verbose=True, score=True)

In [2]:
df = pd.read_excel("thousand_lines_truecased.xlsx", engine="openpyxl",
                  index_col='Unnamed: 0').dropna(subset=['headline'])

In [4]:
df.head()

,id,title_orig_case,content,source,headline
58347,52354,Seattle synagogue vandalism denies Holocaust,[ (CNN)A synagogue in Seattle has become the l...,AllTheNewsComponentsOne,Seattle synagogue vandalism DENIES holocaust
71341,65485,Ford just invested $1 billion in a secretive A...,"['', 'Ford is investing $1 billion in a secret...",AllTheNewsComponentsOne,Ford just invested$ 1 billion in a secretive a...
55118,49079,'Star Wars' Actress Carrie Fisher Dies at Age ...,"Carrie Fisher, the actress best known for play...",AllTheNewsComponentsOne,' Star wars' actress Carrie Fisher dies at age...
102753,108886,Here Are All Of The Victims In The Orlando Ni...,[' At least 50 people were killed and 53 injur...,AllTheNewsComponentsOne,Here are all of the victims in the Orlando nig...
328613,168132,"Despite The Math, Bernie Sanders Has Already Won",Bernie Sanders scored victories Saturday in ...,AllTheNewsKaggle,"Despite the math, Bernie Sanders has already won"


In [6]:
headlines = df['headline'].sample(n=100, random_state=42)

In [11]:
%%time
df_out = humourizer.vandalize_headlines(headlines, return_pandas=True)

Making predictions...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 621/621 [07:35<00:00,  1.36it/s]


Scoring examples...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 635/635 [09:38<00:00,  1.10it/s]


Wall time: 21min 11s


А с BERTSelector'ом?

In [4]:
token_selector = BERTTokenSelector(bert_selector_model='bert-large-uncased-whole-word-masking',
                                   bert_tokenizer='bert-large-uncased-whole-word-masking',
                                   colloc_thresh=0.3, verbose=True)
humourizer = Humourizer(token_selector, token_replacer, verbose=True, score=True)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
%%time
df_out = humourizer.vandalize_headlines(headlines, return_pandas=True)

  2%|█▎                                                                            | 17/999 [01:13<1:10:44,  4.32s/it]


KeyboardInterrupt: 

In [12]:
df_out.columns

Index(['headline', 'masked', 'tokenized', 'span_index', 'span', 'predicted',
       'new span', 'predicted_score'],
      dtype='object')

In [13]:
df_out['predicted_score'].mean()

0.04627715354510295

In [17]:
for i in np.arange(0, 5, 0.5):
    print(i)

0.0
0.5
1.0
1.5
2.0
2.5
3.0
3.5
4.0
4.5


Долго ждать - часа 4...

In [7]:
%%time
cross_tab = []

for i in np.arange(0, 10, 0.5):
    token_selector = CollocateTokenSelector('CM_SpaCy_truecased', thresh=i)
    humourizer = Humourizer(token_selector, token_replacer, verbose=True, score=True)
    df_out = humourizer.vandalize_headlines(headlines, return_pandas=True)
    cross_tab.append({'colloc_thresh':i, 'mean_colbert_score': df_out['predicted_score'].mean(),
                      'n': len(df_out['predicted'].notna()),
                       '0.5': df_out['predicted_score'].quantile(0.5),
                       '0.6': df_out['predicted_score'].quantile(0.6),
                       '0.7': df_out['predicted_score'].quantile(0.7),
                       '0.8': df_out['predicted_score'].quantile(0.8),
                       '0.9': df_out['predicted_score'].quantile(0.9)})

Making predictions...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 73/73 [00:52<00:00,  1.40it/s]


Scoring examples...


  0%|                                                                                                                                                                  | 0/68 [00:00<?, ?it/s]

Making predictions...


...

Making predictions...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [01:01<00:00,  4.73s/it]


Scoring examples...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:24<00:00,  1.75s/it]


Wall time: 2h 4min 19s


In [8]:
cross_tab

[{'colloc_thresh': 0.0,
  'mean_colbert_score': 0.034910054397423666,
  'n': 100,
  '0.5': 1.1549821465450805e-05,
  '0.6': 1.2704665823548567e-05,
  '0.7': 1.417527246303507e-05,
  '0.8': 2.3697155120316894e-05,
  '0.9': 0.0003856647119391761},
 {'colloc_thresh': 0.5,
  'mean_colbert_score': 0.034978800929095104,
  'n': 99,
  '0.5': 1.1536075362528209e-05,
  '0.6': 1.2787745254172476e-05,
  '0.7': 1.4262633885664399e-05,
  '0.8': 2.266867668367923e-05,
  '0.9': 0.00020716196158900865},
 {'colloc_thresh': 1.0,
  'mean_colbert_score': 0.034978800929095104,
  'n': 99,
  '0.5': 1.1536075362528209e-05,
  '0.6': 1.2787745254172476e-05,
  '0.7': 1.4262633885664399e-05,
  '0.8': 2.266867668367923e-05,
  '0.9': 0.00020716196158900865},
 {'colloc_thresh': 1.5,
  'mean_colbert_score': 0.034978799495235365,
  'n': 99,
  '0.5': 1.1518969586177263e-05,
  '0.6': 1.2787745254172476e-05,
  '0.7': 1.4262633885664399e-05,
  '0.8': 2.266867668367923e-05,
  '0.9': 0.00020716196158900865},
 {'colloc_thresh

In [5]:
token_selector = CollocateTokenSelector('CM_SpaCy_truecased', colloc_thresh=-5)
token_replacer = BERTReplacer(bert_model_path='bert_masked_lm_full_model',
                             bert_tokenizer="bert-large-uncased-whole-word-masking",
                             verbose=True, k=3)
humourizer = Humourizer(token_selector, token_replacer, verbose=True, score=True)

In [6]:
df_out = humourizer.vandalize_headlines(headlines, return_pandas=True)

Making predictions...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 64/64 [00:45<00:00,  1.41it/s]


Scoring examples...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 65/65 [01:27<00:00,  1.35s/it]


In [7]:
df_out["predicted_score"].mean()

0.03627962686207866